# LeNet

On MNIST

In [12]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import torch.optim as optim
from tqdm import tqdm


from src.models import LeNet, MiniLeNet
import src.util

In [2]:
# Select Device
use_cuda = True and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else 'cpu')

# Load models
model_init = torch.load('../saves/LeNet_initial_model.ptmodel', weights_only=False)
model_pruned_retrained = torch.load('../saves/LeNet_model_after_retraining.ptmodel', weights_only=False)
model_weight_shared = torch.load('../saves/LeNet_model_after_weight_sharing.ptmodel', weights_only=False)
model_decoded = torch.load('../saves/LeNet_model_after_decoding.ptmodel', weights_only=False)


In [3]:
model_init

LeNet(
  (fc1): MaskedLinear(in_features=784, out_features=300, bias=True)
  (fc2): MaskedLinear(in_features=300, out_features=100, bias=True)
  (fc3): MaskedLinear(in_features=100, out_features=10, bias=True)
)

In [4]:
print("Initial LeNet's parameter number :",sum(p.numel() for p in model_init.parameters()))

path = os.path.expanduser('../saves/LeNet_initial_model.ptmodel')
size_mb_init = os.path.getsize(path) / 1024**2
print(f"Storage space needed by initial LeNet : {size_mb_init:.2f} MB")

Initial LeNet's parameter number : 532810
Storage space needed by initial LeNet : 2.04 MB


In [5]:
print(f"{'Layer':30} {'Params':>12} {'Memory (MB)':>15}")
print("-" * 60)

total_mem = 0

for name, param in model_init.named_parameters():
    num_params = param.numel()
    mem_mb = num_params * param.element_size() / 1024**2
    total_mem += mem_mb
    print(f"{name:30} {num_params:12,d} {mem_mb:15.2f}")

print("-" * 60)
print(f"{'TOTAL':30} {'':12} {total_mem:15.2f} MB")

Layer                                Params     Memory (MB)
------------------------------------------------------------
fc1.weight                          235,200            0.90
fc1.mask                            235,200            0.90
fc1.bias                                300            0.00
fc2.weight                           30,000            0.11
fc2.mask                             30,000            0.11
fc2.bias                                100            0.00
fc3.weight                            1,000            0.00
fc3.mask                              1,000            0.00
fc3.bias                                 10            0.00
------------------------------------------------------------
TOTAL                                                  2.03 MB


In [6]:
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dir_names, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # Vérifie si le fichier existe (évite les liens brisés)
            if os.path.exists(fp):
                total_size += os.path.getsize(fp)
    return total_size

path = os.path.expanduser('../LeNet_encodings/')
size_bytes = get_folder_size(path)
size_kb = size_bytes / 1024
size_mb = size_bytes / (1024 * 1024)

print(f"Storage space needed by encoded LeNet : {size_kb:.2f} kB ({size_mb:.2f} MB)")


Storage space needed by encoded LeNet : 25.55 kB (0.02 MB)


In [7]:
print(f"LeNet compression rate with Pruning, Quantization and Huffman coding : {size_mb_init/size_mb:.1f}x")

LeNet compression rate with Pruning, Quantization and Huffman coding : 81.7x


MiniLeNet's parameter number : 6442
Storage space needed by mini LeNet : 29.06 kB


In [16]:
mean = {'LeNet' : (0.1307,)}
std  = {'LeNet' : (0.3081,)}

def train(model, epochs):
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean['LeNet'], std['LeNet'])
                    ])),
        batch_size=100, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0001)

    model.train()
    for epoch in range(epochs):
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for batch_idx, (data, target) in pbar:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()

            # zero-out all the gradients corresponding to the pruned connections
            for name, p in model.named_parameters():
                if 'mask' in name:
                    continue
                tensor = p.data.cpu().numpy()
                grad_tensor = p.grad.data.cpu().numpy()
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                p.grad.data = torch.from_numpy(grad_tensor).to(device)

            optimizer.step()
            if batch_idx % 10 == 0:
                done = batch_idx * len(data)
                percentage = 100. * batch_idx / len(train_loader)
                pbar.set_description(f'Train Epoch: {epoch} [{done:5}/{len(train_loader.dataset)} ({percentage:3.0f}%)]  Loss: {loss.item():.6f}')

In [ ]:
def test(model, use_cuda=True):
    kwargs = {'num_workers': 5, 'pin_memory': True} if use_cuda else {}
    device = torch.device("cuda" if use_cuda else 'cpu')


    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean['LeNet'], std['LeNet'])
                    ])),
        batch_size=1000, shuffle=False, **kwargs)
    
    
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')
    return accuracy

In [17]:
mini_model = MiniLeNet(mask=False).to(device)
print("MiniLeNet's parameter number :",sum(p.numel() for p in mini_model.parameters()))

train(mini_model, epochs=100)
torch.save(mini_model, '../saves/LeNet_mini.ptmodel')

path = os.path.expanduser('../saves/LeNet_mini.ptmodel')
size_kb_mini = os.path.getsize(path) / 1024
print(f"Storage space needed by mini LeNet : {size_kb_mini:.2f} kB")

MiniLeNet's parameter number : 6442


Train Epoch: 0 [59000/60000 ( 98%)]  Loss: 0.246136: 100%|██████████| 600/600 [00:09<00:00, 63.08it/s]
Train Epoch: 1 [59000/60000 ( 98%)]  Loss: 0.319840: 100%|██████████| 600/600 [00:09<00:00, 64.02it/s]
Train Epoch: 2 [59000/60000 ( 98%)]  Loss: 0.217113: 100%|██████████| 600/600 [00:12<00:00, 47.43it/s]
Train Epoch: 3 [59000/60000 ( 98%)]  Loss: 0.083715: 100%|██████████| 600/600 [00:11<00:00, 51.95it/s]
Train Epoch: 4 [59000/60000 ( 98%)]  Loss: 0.349848: 100%|██████████| 600/600 [00:09<00:00, 64.90it/s]
Train Epoch: 5 [59000/60000 ( 98%)]  Loss: 0.343872: 100%|██████████| 600/600 [00:09<00:00, 65.54it/s]
Train Epoch: 6 [59000/60000 ( 98%)]  Loss: 0.172695: 100%|██████████| 600/600 [00:10<00:00, 58.80it/s]
Train Epoch: 7 [59000/60000 ( 98%)]  Loss: 0.172233: 100%|██████████| 600/600 [00:09<00:00, 61.05it/s]
Train Epoch: 8 [59000/60000 ( 98%)]  Loss: 0.272068: 100%|██████████| 600/600 [00:09<00:00, 63.66it/s]
Train Epoch: 9 [59000/60000 ( 98%)]  Loss: 0.407102: 100%|██████████| 600

Storage space needed by mini LeNet : 29.06 kB


In [18]:
accuracy_init = test(model_init, use_cuda=use_cuda)
accuracy_mini = test(mini_model, use_cuda=use_cuda)

Test set: Average loss: 0.1704, Accuracy: 9580/10000 (95.80%)
Test set: Average loss: 0.2859, Accuracy: 9180/10000 (91.80%)
